# Definition of neuron angle and network phase

In this notebook, we explore the definition of individual neurons angles and network phase in time.

In [ ]:
%matplotlib widget
import lotr.plotting as pltltr
import numpy as np
from lotr import A_FISH, FIGURES_LOCATION, LotrExperiment, dataset_folders
from lotr.notebook_utils import print_source
from lotr.pca import pca_and_phase
from lotr.result_logging import ResultsLogger

COLS = pltltr.COLS
logger = ResultsLogger()

from matplotlib import pyplot as plt
from scipy.stats import wilcoxon
from sklearn.decomposition import PCA
from tqdm import tqdm

## Quantify number of ring neurons

In [ ]:
len(dataset_folders)

In [ ]:
n_rois = [LotrExperiment(f).n_hdns for f in dataset_folders]

logger.add_entry("n_ring_neurons", n_rois, dataset_folders, moment="median", units="n")

## PC over population and network trajectory in phase space

First of all, we load an experiment and we calculate principal components to observe trajectory over time in a reduced number of dimensions.

In [ ]:
exp = LotrExperiment(A_FISH)

t_end = 1500  # crop to half experiment for clarity
pca = PCA(n_components=8)
pca_scores = pca.fit_transform(exp.traces[: t_end * exp.fn, exp.hdn_indexes])

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
s = pltltr.color_plot(
    pca_scores[:, 0],
    pca_scores[:, 1],
    c=np.arange(pca_scores.shape[0]) / exp.fn,
    ax=ax,
    cmap=COLS["time"],
    lw=2,
)
pltltr.add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=5, ylen=5)

pltltr.add_cbar(
    s,
    ax,
    inset_loc=(0.85, 0.95, 0.2, 0.04),
    ticks=(0, t_end - 100),
    ticklabels=(0, t_end),
    orientation="horizontal",
    label="Time (s)",
    labelsize=8,
)
ax.axis("equal")
plt.show()

pltltr.savefig("population_trajectory")

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.0))
ax.bar(
    np.arange(pca.n_components) + 1,
    pca.explained_variance_ratio_,
    fc=COLS["ring"],
    label="components",
)
ax.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(pca.explained_variance_ratio_),
    label="cumulative",
    c=(0.3,) * 3,
    zorder=100,
)

pltltr.despine(ax)
ax.legend()
ax.set(xlabel="n. of components", ylabel="Variance", ylim=(0, 1.1), xticks=[2, 4, 6, 8])
plt.tight_layout()

pltltr.savefig("variance_expl_pop_onefish")

Yay! This looks intriguingly circular! What is happening there?

### Variance explained with PCA over pouplation
First, we check the variance explained by this decomposition

In [ ]:
# Find principal components from selected neurons. The time slice cuts out trace extrema and
# periods of vigorous directional motion in experiments that have it.

all_var_expl_ring = []
all_var_expl_nonring = []
for path in dataset_folders:
    exp = LotrExperiment(path)

    # Loop over data and control:
    for data_list, roi_selection in zip(
        [all_var_expl_ring, all_var_expl_nonring], [exp.hdn_indexes, exp.rndcnt_indexes]
    ):
        pca = PCA(n_components=15).fit(exp.traces[exp.pca_t_slice, roi_selection])
        data_list.append(pca.explained_variance_ratio_)

all_var_expl_ring = np.array(all_var_expl_ring).T
all_var_expl_nonring = np.array(all_var_expl_nonring).T

In [ ]:
tot_var_expl_ring, tot_var_expl_nonring = [
    all_var_expl[:2, :].sum(0)
    for all_var_expl in [all_var_expl_ring, all_var_expl_nonring]
]

step = 0.05

f, ax = plt.subplots(figsize=(2.5, 2), gridspec_kw=dict(bottom=0.2, left=0.2, top=0.8))

ax.hist(tot_var_expl_ring, np.arange(0, 1, step), label="r1π ROIs", fc=COLS["ring"])
ax.hist(
    tot_var_expl_nonring,
    np.arange(0, 1, step),
    label="cnt ROIs",
    histtype="step",
    ec=".3",
)

medians = []
for values, col in zip([tot_var_expl_nonring, tot_var_expl_ring], [".3", COLS["ring"]]):
    medians.append(np.median(values))
    ax.axvline(medians[-1], linestyle="dashed", c=pltltr.dark_col(col))
print(wilcoxon(tot_var_expl_ring, tot_var_expl_nonring))
ax.text(
    min(medians) + abs((medians[0] - medians[1])) / 2,
    18,
    pltltr.get_pval_stars(wilcoxon(tot_var_expl_ring, tot_var_expl_nonring)),
    fontsize=8,
)


ax.set(
    xlim=(0, 1.1),
    xlabel="var. expl. by first 2 PCs (pop.)",
    ylabel=("fish count"),
    yticks=(5, 10, 15, 20, 25),
)
# plt.tight_layout()
ax.legend(bbox_to_anchor=(0, 0, 1, 1), bbox_transform=f.transFigure)
pltltr.despine(ax)

plt.show()
pltltr.savefig("pcapop_var_expl", folder="S2")

In [ ]:
wilcoxon(tot_var_expl_ring, tot_var_expl_nonring)

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2), gridspec_kw=dict(bottom=0.2, left=0.2, top=0.8))
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_nonring, 0),
    lw=1,
    c=".7",
    label="__nolegend__",
)
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_nonring, 0).mean(1),
    lw=1.5,
    c=".3",
    label="cnt ROIs",
)

plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_ring, 0),
    lw=1,
    c=pltltr.shift_lum(COLS["ring"], 0.4),
    label="__nolegend__",
)
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_ring, 0).mean(1),
    lw=1.5,
    c=COLS["ring"],
    label="r1π ROIs",
)

ax.set(
    ylim=(0, 1.1),
    title="PCA over population",
    xlabel="N. of components",
    ylabel="Cum. variance expl.",
)
ax.legend()  # loc=2, bbox_to_anchor=(0.5, 0, 0.1, 0.2))
pltltr.despine(ax)
plt.show()
pltltr.savefig("pcapop_cumsum_var", folder="S2")

#### Log variance explained over pop

In [ ]:
logger.add_entry(
    "pcapop_variance_explained",
    tot_var_expl_ring,
    dataset_folders,
    moment="median",
    units="fract.",
)

## PCA over time and neuron angles

If we now look at PCs computed over time, we get a similar circular pattern:

In [ ]:
exp = LotrExperiment(A_FISH)

# Find principal components from selected neurons:
pca = PCA(n_components=8).fit(exp.traces[:, exp.hdn_indexes].T)

# Compute scores for all neurons:
pca_scores_t = pca.transform(exp.traces[:, :].T)

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
s = ax.scatter(
    pca_scores_t[exp.hdn_indexes, 0],
    pca_scores_t[exp.hdn_indexes, 1],
    color=COLS["ring"],
    label="r1π ROIs",
    lw=0.2,
)
s = ax.scatter(
    pca_scores_t[exp.nonhdn_indexes, 0],
    pca_scores_t[exp.nonhdn_indexes, 1],
    label="cnt ROIs",
    fc="none",
    ec="k",
    lw=0.2,
    zorder=-100,
)

plt.legend(
    frameon=False,
    bbox_to_anchor=(1.1, -0.1, 0.04, 0.2),
    markerscale=1,
    handletextpad=-0.3,
    fontsize=8,
)
pltltr.add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
ax.axis("equal")
# plt.tight_layout()
plt.subplots_adjust(bottom=0.2)
plt.show()

pltltr.savefig("individual_rois_pcs_cnt")

This is interesting! How much variability of the data are we reconstructing with those two PCs over which cells are circularly distributed?

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.0))
ax.bar(
    np.arange(pca.n_components) + 1,
    pca.explained_variance_ratio_,
    fc=COLS["ring"],
    label="components",
)
ax.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(pca.explained_variance_ratio_),
    label="cumulative",
    c=(0.3,) * 3,
    zorder=100,
)

l_h = 0.85
ax.axhline(
    l_h, linestyle="dashed", c=(0.7,) * 3,
)
ax.text(8.5, l_h - 0.05, f"{l_h:0.2f}", c=(0.7,) * 3, ha="right", va="top", fontsize=8)
pltltr.despine(ax)
ax.legend()
ax.set(xlabel="n. of components", ylabel="Variance", ylim=(0, 1.1), xticks=[2, 4, 6, 8])
plt.tight_layout()

pltltr.savefig("variance_expl_onefish")

### Variance explained in the entire population

First let's make some fancy plot for all fish

In [ ]:
all_pc_scores = []
all_pc_scores_allrois = []
all_angles = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    pca_scores_t, roi_pc_angles, _, circle_params = pca_and_phase(
        exp.traces[exp.pca_t_slice, exp.hdn_indexes].T
    )
    # normalize to circle for practicity:
    all_pc_scores.append((pca_scores_t[:, :2] - circle_params[:2]) / circle_params[2])
    all_angles.append(roi_pc_angles)

    pca_scores_t, roi_pc_angles, _, _ = pca_and_phase(
        exp.traces[exp.pca_t_slice, exp.hdn_indexes].T, exp.traces[exp.pca_t_slice, :].T
    )
    all_pc_scores_allrois.append(
        (pca_scores_t[:, :2] - circle_params[:2]) / circle_params[2]
    )

In [ ]:
n_cols = 8
f, axs = plt.subplots(4, n_cols, figsize=(7, 3))
for i, (pc_scores, pc_scores_all, angles) in enumerate(
    zip(all_pc_scores, all_pc_scores_allrois, all_angles)
):
    ax = axs[i // n_cols, i % n_cols]
    ax.plot(*pltltr.get_circle_xy((0, 0, 1)), c=(0.2,) * 3, lw=0.5, zorder=-100)
    ax.scatter(
        pc_scores_all[:, 0],
        pc_scores_all[:, 1],
        s=3,
        fc="none",
        ec="k",
        lw=0.05,  # c=angles, cmap=COLS["phase_light"]
        rasterized=True,
    )
    ax.scatter(
        pc_scores[:, 0],
        pc_scores[:, 1],
        s=3,
        color=COLS["ring"],  # c=angles, cmap=COLS["phase_light"]
        rasterized=True,
    )
    ax.axis("equal")

for ax in axs.flatten():
    pltltr.despine(ax, "all")

pltltr.add_scalebar(
    axs[-1, 0],
    xlabel="PC1",
    ylabel="PC2",
    xlen=0.8,
    ylen=0.8,
    line_params=dict(lw=0.5),
    text_params=dict(fontsize=6),
)

pltltr.savefig("rois_angles_allfish", folder="S2")

We can now try to have a look in the full dataset how much variance the first two principal components explain. We will compare this against the distribution of variance explained by first two components in the rest of the dataset.

In [ ]:
# Find principal components from selected neurons. The time slice cuts out trace extrema and
# periods of vigorous directional motion in experiments that have it.

all_var_expl_ring = []
all_var_expl_nonring = []
for path in dataset_folders:
    exp = LotrExperiment(path)

    # Loop over data and control:
    for data_list, roi_selection in zip(
        [all_var_expl_ring, all_var_expl_nonring], [exp.hdn_indexes, exp.rndcnt_indexes]
    ):
        pca = PCA(n_components=15).fit(exp.traces[exp.pca_t_slice, roi_selection].T)
        data_list.append(pca.explained_variance_ratio_)

all_var_expl_ring = np.array(all_var_expl_ring).T
all_var_expl_nonring = np.array(all_var_expl_nonring).T

In [ ]:
tot_var_expl_ring, tot_var_expl_nonring = [
    all_var_expl[:2, :].sum(0)
    for all_var_expl in [all_var_expl_ring, all_var_expl_nonring]
]

step = 0.05

f, ax = plt.subplots(figsize=(2.5, 2), gridspec_kw=dict(bottom=0.2, left=0.2, top=0.8))

ax.hist(tot_var_expl_ring, np.arange(0, 1, step), label="r1π ROIs", fc=COLS["ring"])
ax.hist(
    tot_var_expl_nonring,
    np.arange(0, 1, step),
    label="cnt ROIs",
    histtype="step",
    ec=".3",
)

medians = []
for values, col in zip([tot_var_expl_nonring, tot_var_expl_ring], [".3", COLS["ring"]]):
    medians.append(np.median(values))
    ax.axvline(medians[-1], linestyle="dashed", c=pltltr.dark_col(col))

ax.text(
    np.median(medians[-1]) + 0.02,
    18,
    f"med: {medians[-1]:0.2f}",
    c=pltltr.dark_col(col),
    fontsize=8,
)

ax.text(
    min(medians) + abs((medians[0] - medians[1])) / 2,
    18,
    pltltr.get_pval_stars(wilcoxon(tot_var_expl_ring, tot_var_expl_nonring)),
    fontsize=8,
)
print(wilcoxon(tot_var_expl_ring, tot_var_expl_nonring))


ax.set(
    xlim=(0, 1.1),
    xlabel="var. expl. by first 2 PCs (time)",
    ylabel=("fish count"),
    yticks=(5, 10, 15, 20, 25),
)
# plt.tight_layout()
ax.legend(bbox_to_anchor=(0, 0, 1, 1), bbox_transform=f.transFigure)
pltltr.despine(ax)

plt.show()
pltltr.savefig("pcatime_var_expl", folder="S2")

In [ ]:
wilcoxon(tot_var_expl_ring, tot_var_expl_nonring)

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2), gridspec_kw=dict(bottom=0.2, left=0.2, top=0.8))
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_ring, 0),
    lw=1,
    c=pltltr.shift_lum(COLS["ring"], 0.4),
    label="__nolegend__",
)
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_ring, 0).mean(1),
    lw=1.5,
    c=COLS["ring"],
    label="r1π ROIs",
)
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_nonring, 0),
    lw=1,
    c=".7",
    label="__nolegend__",
)
plt.plot(
    np.arange(pca.n_components) + 1,
    np.cumsum(all_var_expl_nonring, 0).mean(1),
    lw=1.5,
    c=".3",
    label="cnt ROIs",
)

ax.set(
    ylim=(0, 1.1),
    title="PCA over time",
    xlabel="N. of components",
    ylabel="Cum. variance expl.",
)
ax.legend()  # loc=2, bbox_to_anchor=(0.5, 0, 0.1, 0.2))
pltltr.despine(ax)
plt.show()
pltltr.savefig("pcatime_cumsum_var", folder="S2")

In [ ]:
logger.add_entry(
    "pcatime_variance_explained",
    tot_var_expl_ring,
    dataset_folders,
    moment="median",
    units="fract.",
)

## Neuron angles

On these data, it is simple to fit a circle and define for every ROI a "PC angle" as the angle from the center of such circle. 
For $i^{th}$ ROI,

$$ 
{PCangle}_i = arctan(\frac{PC1_i - center_x}{PC2_i - center_y}) 
$$

For this, we use the `pca_and_phase` function:

In [ ]:
print_source(pca_and_phase)

In [ ]:
# let's get again our example:
exp = LotrExperiment(A_FISH)

pca_scores_t, roi_pc_angles, _, circle_params = pca_and_phase(
    exp.traces[:, exp.hdn_indexes].T
)

# To simplify plotting, we just center the circle on 0:
pca_scores_t[:, :2] = pca_scores_t[:, :2] - circle_params[:2]

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.5))
s = ax.scatter(
    pca_scores_t[:, 0],
    pca_scores_t[:, 1],
    c=roi_pc_angles,
    cmap=COLS["phase_light"],
    label="r1π ROIs",
    lw=0.2,
)
ax.plot(*pltltr.get_circle_xy((0, 0, circle_params[2])), c=(0.2,) * 3, lw=1)
pltltr.add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
ax.axis("equal")

pltltr.add_cbar(
    s,
    ax,
    (0.92, 1.1, 0.2, 0.04),
    ticks=(-np.pi + 0.2, 0, np.pi - 0.2),
    ticklabels=("-π", 0, "π"),
    label="PC angle",
    orientation="horizontal",
)
plt.show()

pltltr.savefig("rois_angles")

Now, make a fancy plot for all fish:

## From neuron angles to network phase

Now, we can start from the fit of neuron angles in PC space to get to a reasonable notion of "phase" in time of the network. At every timepoint, we will take an average vector sum over the neurons of the circle, weighted by the intensity of their fluorescence. At every timepoint, the fluorescence intensity is normalized to have 0 sum. This mean that a neuron can have negative weight and hence negative contribution. Zero-sum normalization help reducing biases that might come from non homogeneous distribution of ROIs along the circle. 

In [ ]:
from lotr.rpca_calculation import get_zero_mean_weights
from lotr.utils import get_vect_angle

# let's get again our example:
exp = LotrExperiment(A_FISH)

norm_activity = get_zero_mean_weights(exp.traces[:, exp.hdn_indexes].T).T
avg_vects = np.einsum("ij,ik->jk", norm_activity.T, pca_scores_t[:, :2])

network_phase = get_vect_angle(avg_vects.T)

In [ ]:
scale_arr = 5000
scale_mn = 60
f_lim = 0.01
f, axs = plt.subplots(1, 3, figsize=(6, 2.5))
for n, (i, ax) in enumerate(zip([900, 1000, 3400], axs)):
    s = ax.scatter(
        pca_scores_t[:, 0],
        pca_scores_t[:, 1],
        c=norm_activity[i, :],
        cmap=COLS["dff_opp"],
        vmin=-f_lim,
        vmax=f_lim,
        lw=0.0,
    )

    for r in range(len(exp.hdn_indexes)):
        angle, length = roi_pc_angles[r], norm_activity[i, r] * scale_arr
        ax.plot(
            [0, np.cos(angle) * length],
            [0, np.sin(angle) * length],
            c=(0.7,) * 3,
            lw=0.2,
        )

    angle, length = network_phase[i], scale_mn
    ax.plot(
        [0, np.cos(angle) * length],
        [0, np.sin(angle) * length],
        c=pltltr.get_default_phase_col(network_phase[i]),
        lw=1,
    )

    # timestamp:
    ax.text(-100, 100, f"t = {i / exp.fn:3.0f} s", fontsize=8)

    if n == 0:
        pltltr.add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
        cbar = pltltr.add_cbar(
            s,
            ax,
            (1.1, 0.2, 0.2, 0.04),
            ticks=(-f_lim + 0.001, f_lim - 0.001),
            ticklabels=("-", "+"),
            titlesize=8,
            title="ΔF weight",
            orientation="horizontal",
        )
    else:  # dummy for scaling:
        pltltr.add_scalebar(
            ax, xlabel="", ylabel="", xlen=30, ylen=30, line_params=dict(lw=0)
        )

    ax.axis("equal")

pltltr.savefig("network_phase_explanation", folder="S4")

In [ ]:
from lotr.plotting.gifs_gen.network_phase import network_phase_animation

In [ ]:
# Create animation to explore the concept.
# It won't be displayed inline, and GitHub actions could complay if we try to save mp4, hence the try:
try:
    f = network_phase_animation(frames=list(range(5, 4000, 5)))
except ValueError:
    pass
# plt.show()

## Network phase and trajectory in phase space

We can now look again at the network trajectory in phase space, to see what is the relationship between the network phase defined above and positions in phase space.

In [ ]:
rasterized = True
f, ax = plt.subplots(figsize=(2.5, 2.5))
s = pltltr.color_plot(
    pca_scores[:, 0],
    pca_scores[:, 1],
    c=network_phase[: t_end * exp.fn],
    ax=ax,
    cmap=COLS["phase"],
    lw=2,
    rasterized=True,
)
pltltr.add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=5, ylen=5)

pltltr.add_phase_cbar(
    s, ax, inset_loc=(0.85, 0.95, 0.2, 0.04), orientation="horizontal",
)
ax.axis("equal")
plt.show()

rast = "rast" if rasterized else ""
pltltr.savefig("population_trajectory_with_phase" + rast, folder="S4")

Cool! It seems that our definition of phase really captures what is happening in this trajectory.

It would be very cool to see whether this organization is linked to anatomical disposition of cells. This will be analyzed in the next notebook.

In [ ]:
%%time

import matplotlib.gridspec as gridspec

# gridspec inside gridspec
fig = plt.figure(figsize=(8, 12))

gs0 = gridspec.GridSpec(16, 2, figure=fig, bottom=0.02, left=0.02, top=0.98, right=0.98)

# the following syntax does the same as the GridSpecFromSubplotSpec call above:
for i in range(gs0.nrows):
    for j in range(gs0.ncols):
        try:
            path = dataset_folders[i * 2 + j]

            try:
                exp = LotrExperiment(path)
            except IndexError:
                continue

            gs01 = gs0[i, j].subgridspec(1, 5)

            ax_pca = fig.add_subplot(gs01[0])
            ax_traces = fig.add_subplot(gs01[1:])

            pca = PCA(n_components=8)
            pca_scores = pca.fit_transform(exp.traces[exp.pca_t_slice, exp.hdn_indexes])

            sorted_traces = exp.traces[:, exp.hdn_indexes][
                :, np.argsort(exp.rpc_angles)
            ]
            phase = exp.network_phase.copy()
            phase[1:][np.abs(np.diff(phase)) > np.pi] = np.nan

            im = ax_traces.imshow(
                sorted_traces[exp.pca_t_slice, :].T,
                extent=[exp.pca_t_lims[0], exp.pca_t_lims[1], 0, exp.n_hdns],
                aspect="auto",
                cmap=COLS["dff_plot"],
                vmin=-1.7,
                vmax=2.0,
            )
            ax_traces.plot(
                exp.time_arr[exp.pca_t_slice],
                (phase[exp.pca_t_slice] / np.pi + 1) * exp.n_hdns / 2,
                c=COLS["ph_plot"],
                lw=1,
                label="Network phase",
                rasterized=True,
            )

            s = pltltr.color_plot(
                pca_scores[:, 0],
                pca_scores[:, 1],
                c=exp.network_phase[exp.pca_t_slice],
                ax=ax_pca,
                cmap=COLS["phase"],
                lw=0.5,
                rasterized=True,
            )

            # ax_pca.plot(pca_scores[:, 0], pca_scores[:, 1], lw=0.5)
            ax_pca.axis("equal")
            pltltr.add_scalebar(ax_pca, xlabel="", ylabel="", xlen=5, ylen=5)

            pltltr.add_scalebar(
                ax_traces,
                xlabel="",
                ylabel="",
                ypos=-2,
                xpos=exp.pca_t_lims[0],
                xlen=100,
                ylen=0,
            )
            ax_traces.set_title(exp.exp_code, loc="left", fontsize=6, pad=0)

            pltltr.despine(ax_traces, "all")

            if i == 0 and j == 0:
                pltltr.add_dff_cbar(
                    im,
                    ax_traces,
                    (1.05, 0.15, 0.02, 0.3),
                    titlesize=6,
                    labelsize=5,
                    ticklabels=None,
                )
                ax_traces.legend(
                    loc=2,
                    bbox_to_anchor=(0.7, 1.3),
                    fontsize=7,
                    labelcolor="linecolor",
                    handlelength=0.0,
                )

                pltltr.add_phase_cbar(
                    s,
                    ax_traces,
                    inset_loc=(-0.15, 1.1, 0.1, 0.1),
                    orientation="horizontal",
                    titlesize=7,
                    labelsize=6,
                )
        except IndexError:
            pass
plt.show()
# pltltr.add_scalebar(axs[0, 0], xlabel="PC1", ylabel="PC2", xlen=5, ylen=5, fontsize=6)
pltltr.savefig("all_traces_with_phase" + rast, folder="S4bis")